# Live Bitcoin Feed Playground (BTC/USD:CXTALP)
Self-contained notebook to: (1) initialize config & connections, (2) subscribe to the live Bitcoin symbol (active on weekends), (3) stream quotes & recent candles, (4) explore frames, (5) cleanly shutdown.

**Pipeline Touchpoints**: DXLink → Redis (pub/sub + cache) + Telegraf (HTTP) → (optionally) Influx. This notebook goes directly to DXLink (bypassing FastAPI edge) for lowest latency validation.

**Stop / Rerun Guidance**: If a cell is interrupted mid-loop, re-run the subscription cell before restarting the watcher. Always run the shutdown cell before closing the notebook kernel.

In [1]:
import asyncio, logging, os, math, time, textwrap
from datetime import datetime
import pandas as pd
from IPython.display import display, clear_output, Markdown

from tastytrade.common.logging import setup_logging
from tastytrade.config import RedisConfigManager
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.messaging.processors import (
    TelegrafHTTPEventProcessor,
    RedisEventProcessor,
)
from tastytrade.config.enumerations import Channels

# Pandas display tuning for richer exploratory output
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()
setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix="dev_tastytrade",
    console=True,
    file=True,
)
logging.getLogger("asyncio").setLevel(logging.WARNING)
logging.info("Environment ready.")

2025-08-09 17:39:22 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250809.log
2025-08-09 17:39:22 - INFO:root:32:Environment ready.
2025-08-09 17:39:22 - INFO:root:32:Environment ready.


## Parameters

In [2]:
# Core symbol (weekend-active crypto)
SYMBOL = "BTC/USD:CXTALP"
# Candle intervals to watch (ordered smallest→largest)
CANDLE_INTERVALS = ["m", "5m", "15m", "1h", "4h", "1d"]
# Backfill start reference (adjust as desired)
BACKFILL_START = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
# Watch loop runtime (seconds); set None for indefinite (Ctrl+C to interrupt)
WATCH_SECONDS = 90
# Refresh period for live panel
REFRESH_SECS = 5
# Forward fill lookback days (for continuity)
FORWARD_FILL_LOOKBACK_DAYS = 3

SYMBOL_EVENT_PREFIX = SYMBOL  # raw symbol used by subscription helpers
logging.info("Parameters set for %s", SYMBOL)

2025-08-09 17:39:22 - INFO:root:15:Parameters set for BTC/USD:CXTALP


## Open Connections & Wire Processors

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)
credentials = Credentials(config=config, env="Live")

# DXLink with Redis-backed subscription store (for replay / state)
dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

# Optional telemetry / distribution processors (Telegraf HTTP + Redis Pub/Sub)
for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())
    handler.add_processor(RedisEventProcessor())

logging.info(
    "DXLink open; processors attached. Channels: %s", list(dxlink.router.handler.keys())
)

2025-08-09 17:39:22 - INFO:tastytrade.config.manager:174:Initialized 19 variables from .env file in Redis
2025-08-09 17:39:23 - INFO:tastytrade.connections.requests:148:Session created successfully
2025-08-09 17:39:23 - INFO:tastytrade.connections.requests:148:Session created successfully
2025-08-09 17:39:23 - INFO:tastytrade.connections.subscription:118:Redis ping response: True
2025-08-09 17:39:23 - INFO:tastytrade.connections.subscription:122:Redis version: 7.4.3
2025-08-09 17:39:23 - INFO:tastytrade.connections.subscription:123:Connected clients: 3
2025-08-09 17:39:23 - INFO:tastytrade.connections.subscription:118:Redis ping response: True
2025-08-09 17:39:23 - INFO:tastytrade.connections.subscription:122:Redis version: 7.4.3
2025-08-09 17:39:23 - INFO:tastytrade.connections.subscription:123:Connected clients: 3
2025-08-09 17:39:23 - INFO:root:14:DXLink open; processors attached. Channels: [<Channels.Control: 0>, <Channels.Quote: 7>, <Channels.Trade: 5>, <Channels.Greeks: 11>, <Cha

2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:225:SETUP
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2025-08-09 17:39:23 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:7
2025-08-09 17:39:23 - INFO:tastytrade.messaging.han

## Subscribe (Ticker + Candles)

In [4]:
# Ticker subscription
await dxlink.subscribe([SYMBOL])

# Candle subscriptions with bounded wait timeout
for iv in CANDLE_INTERVALS:
    coro = dxlink.subscribe_to_candles(
        symbol=SYMBOL, interval=iv, from_time=BACKFILL_START
    )
    await asyncio.wait_for(coro, timeout=15)

logging.info(
    "Subscribed to %s (ticker + %d candle intervals).", SYMBOL, len(CANDLE_INTERVALS)
)

2025-08-09 17:39:23 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP
2025-08-09 17:39:23 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=m}
2025-08-09 17:39:23 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=5m}
2025-08-09 17:39:23 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=15m}
2025-08-09 17:39:23 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=h}
2025-08-09 17:39:23 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=4h}
2025-08-09 17:39:23 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=d}
2025-08-09 17:39:23 - INFO:root:11:Subscribed to BTC/USD:CXTALP (ticker + 6 candle intervals).
2025-08-09 17:39:23 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=m}
2025-08-09 17:39:23 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{

## Forward Fill Recent History (Continuity)

In [5]:
from tastytrade.utils.time_series import forward_fill

for iv in CANDLE_INTERVALS:
    event_symbol = f"{SYMBOL}{{={iv}}}"
    logging.debug("Forward-filling %s", event_symbol)
    forward_fill(symbol=event_symbol, lookback_days=FORWARD_FILL_LOOKBACK_DAYS)
logging.info("Forward fill completed.")

2025-08-09 17:39:23 - INFO:root:7:Forward fill completed.


## Live Watch Panel

In [6]:
async def watch(symbol: str):
    quote_handler = dxlink.router.handler[Channels.Quote].processors["feed"]
    candle_handler = dxlink.router.handler[Channels.Candle].processors["feed"]
    start = time.time()
    iteration = 0
    smallest_iv = CANDLE_INTERVALS[0]
    candle_key = f"{symbol}{{={smallest_iv}}}"

    # helper to find a plausible time-like column
    def detect_time_col(cols):
        candidates = {"time", "timestamp", "ts", "datetime"}
        for c in cols:
            if c.lower() in candidates:
                return c
        return None

    while True:
        now = time.time()
        if WATCH_SECONDS is not None and (now - start) > WATCH_SECONDS:
            logging.info("Watch loop reached %ds, exiting.", WATCH_SECONDS)
            break
        iteration += 1
        clear_output(wait=True)
        display(Markdown(f"### Live Panel — {symbol} (iter {iteration})"))
        # Quote frame (entire)
        qf = quote_handler.pl
        if not qf.is_empty():
            tcol = detect_time_col(qf.columns)
            if tcol:
                try:
                    last_quote = qf.sort(by=tcol).tail(1)
                except Exception as e:
                    logging.debug(
                        "Sort by %s failed: %s; falling back to tail(1)", tcol, e
                    )
                    last_quote = qf.tail(1)
            else:
                last_quote = qf.tail(1)
            display(Markdown("**Last Quote**"))
            display(last_quote)
        else:
            display(Markdown("_No quotes yet._"))
        # Candle frame (focused smallest interval)
        cf = candle_handler.frames.get(candle_key)
        if cf is not None and not cf.is_empty():
            tcol2 = detect_time_col(cf.columns)
            display(Markdown(f"**Recent Candles ({smallest_iv})**"))
            if tcol2:
                try:
                    display(cf.sort(by=tcol2, descending=True).head(8))
                except Exception as e:
                    logging.debug(
                        "Candle sort by %s failed: %s; using tail()", tcol2, e
                    )
                    display(cf.tail(8))
            else:
                display(cf.tail(8))
        else:
            display(Markdown(f"_No candles yet for {candle_key}._"))
        await asyncio.sleep(REFRESH_SECS)


await watch(SYMBOL)

### Live Panel — BTC/USD:CXTALP (iter 18)

**Last Quote**

eventSymbol,bidPrice,askPrice,bidSize,askSize
str,f64,f64,f64,f64
"""BTC/USD:CXTALP""",116150.08,117333.11,0.05,0.05


**Recent Candles (m)**

eventSymbol,time,eventFlags,index,sequence,count,open,high,low,close,volume,bidVolume,askVolume,openInterest,vwap,impVolatility
str,datetime[μs],i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BTC/USD:CXTALP{=m}""",2025-08-09 17:40:00,0,7536641966289715200,0,502,116728.35,116748.7,116711.04,116741.6,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=m}""",2025-08-09 17:39:00,0,7536641708591677440,0,594,116739.33,116746.14,116728.36,116728.36,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=m}""",2025-08-09 17:38:00,0,7536641450893639680,0,939,116755.13,116763.23,116739.25,116739.33,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=m}""",2025-08-09 17:37:00,0,7536641193195601920,0,881,116809.34,116809.49,116748.54,116754.91,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=m}""",2025-08-09 17:36:00,0,7536640935497564160,0,880,116760.45,116815.73,116743.81,116809.34,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=m}""",2025-08-09 17:35:00,0,7536640677799526400,0,543,116709.2,116764.85,116708.97,116760.4,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=m}""",2025-08-09 17:34:00,0,7536640420101488640,0,512,116732.87,116732.87,116704.27,116707.53,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=m}""",2025-08-09 17:33:00,0,7536640162403450880,0,114,116705.62,116734.96,116702.4,116729.96,null,null,null,0.0,null,null


2025-08-09 17:40:54 - INFO:root:20:Watch loop reached 90s, exiting.


## Plot: Recent Candles
Interactive candlestick plot utilities. Run the setup cell, then either a one-shot render or a short live updating loop. Adjust interval or lookback as needed.

In [13]:
import plotly.graph_objects as go
from datetime import timezone, timedelta


def get_candle_frame(symbol: str, interval: str):
    key = f"{symbol}{{={interval}}}"
    candle_handler = dxlink.router.handler[Channels.Candle].processors["feed"]
    frame = candle_handler.frames.get(key)
    if frame is None or frame.is_empty():
        return None
    # Attempt to find time-like column
    for candidate in ["time", "timestamp", "datetime", "ts"]:
        if candidate in frame.columns:
            tcol = candidate
            break
    else:
        # fabricate a positional index if no time column
        frame = frame.with_row_count(name="row_index")
        tcol = "row_index"
    # Ensure sorted ascending
    try:
        frame_sorted = frame.sort(by=tcol)
    except Exception:
        frame_sorted = frame
    return frame_sorted, tcol


interval_for_plot = "5m"  # adjust as desired
lookback_rows = 120  # number of recent rows to display

res = get_candle_frame(SYMBOL, interval_for_plot)
if res is None:
    display(Markdown(f"_No candle data yet for {SYMBOL} {interval_for_plot}_"))
else:
    frame_sorted, tcol = res
    pl_slice = frame_sorted.tail(lookback_rows)
    # Map plausible column names (robust to vendor changes)
    colmap = {}
    for logical, options in {
        "open": ["open", "o"],
        "high": ["high", "h"],
        "low": ["low", "l"],
        "close": ["close", "c", "last"],
        "volume": ["volume", "v"],
    }.items():
        for opt in options:
            if opt in pl_slice.columns:
                colmap[logical] = opt
                break
    missing_ohlc = {k for k in ["open", "high", "low", "close"] if k not in colmap}
    if missing_ohlc:
        display(
            Markdown(
                f"⚠️ Missing OHLC columns: {missing_ohlc}. Raw cols: {pl_slice.columns}"
            )
        )
    df = pl_slice.to_pandas()
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=df[tcol],
                open=df[colmap.get("open")],
                high=df[colmap.get("high")],
                low=df[colmap.get("low")],
                close=df[colmap.get("close")],
                increasing_line_color="#26a69a",
                decreasing_line_color="#ef5350",
                name=f"{SYMBOL} {interval_for_plot}",
            )
        ]
    )
    fig.update_layout(
        title=f"{SYMBOL} {interval_for_plot} — Last {len(df)} Bars",
        xaxis_title="Time",
        yaxis_title="Price",
        template="plotly_dark",
        height=600,
        margin=dict(l=40, r=20, t=60, b=40),
        xaxis_rangeslider_visible=False,
    )
    fig.show()

AttributeError: 'NoneType' object has no attribute 'handler'

## Indicators: MACD & Hull (5m)
Live incremental MACD + Hull using production realtime module helpers (`tastytrade.realtime.notebook`). Run the setup cell below to display a live-updating chart.

In [ ]:
import asyncio
from tastytrade.realtime.notebook import (
    setup_realtime_chart,
    start_chart_stream_task,
)

LIVE_INTERVAL = '5m'
MIN_BOOTSTRAP_ROWS = 40
POLL_SECS = 2.0

chart = await setup_realtime_chart(dxlink, SYMBOL, interval=LIVE_INTERVAL, min_rows=MIN_BOOTSTRAP_ROWS)
fig = chart.figure
fig.update_layout(height=720, title=f"{SYMBOL} {LIVE_INTERVAL} — Live MACD + Hull")
fig.show()
stream_task = start_chart_stream_task(dxlink, chart, SYMBOL, interval=LIVE_INTERVAL, poll_secs=POLL_SECS)
stream_task

In [9]:
# OPTIONAL: short live updating candlestick loop (updates in-place)
import plotly.graph_objects as go
from IPython.display import display

LIVE_UPDATE_INTERVAL = 5  # seconds
LIVE_UPDATE_DURATION = 60  # total seconds (set None for indefinite)


async def live_candles(
    symbol: str, interval: str, update_secs: int = 5, total_secs: int | None = 60
):
    res = get_candle_frame(symbol, interval)
    if res is None:
        display(Markdown(f"_No candle data yet for {symbol} {interval}_"))
        return
    frame_sorted, tcol = res
    df = frame_sorted.tail(lookback_rows).to_pandas()
    fig = go.FigureWidget(
        data=[
            go.Candlestick(
                x=df[tcol],
                open=df[colmap.get("open")] if "open" in colmap else None,
                high=df[colmap.get("high")] if "high" in colmap else None,
                low=df[colmap.get("low")],
                close=df[colmap.get("close")],
                increasing_line_color="#26a69a",
                decreasing_line_color="#ef5350",
                name=f"{symbol} {interval}",
            )
        ]
    )
    fig.update_layout(
        title=f"Live {symbol} {interval}",
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        height=600,
        margin=dict(l=40, r=20, t=60, b=40),
    )
    display(fig)
    start = time.time()
    while True:
        if total_secs is not None and (time.time() - start) > total_secs:
            break
        await asyncio.sleep(update_secs)
        res = get_candle_frame(symbol, interval)
        if res is None:
            continue
        frame_sorted, _ = res
        df = frame_sorted.tail(lookback_rows).to_pandas()
        with fig.batch_update():
            fig.data[0].x = df[tcol]
            fig.data[0].open = df.get(colmap.get("open")) if "open" in colmap else None
            fig.data[0].high = df.get(colmap.get("high")) if "high" in colmap else None
            fig.data[0].low = df.get(colmap.get("low")) if "low" in colmap else None
            fig.data[0].close = (
                df.get(colmap.get("close")) if "close" in colmap else None
            )
            fig.layout.title = (
                f"Live {symbol} {interval} — Updated {datetime.utcnow().isoformat()}"
            )


# Run the live loop (uncomment to execute)
# await live_candles(SYMBOL, interval_for_plot, update_secs=LIVE_UPDATE_INTERVAL, total_secs=LIVE_UPDATE_DURATION)

## Ad-hoc Exploration

In [10]:
# Example: access full quote frame
dxlink.router.handler[Channels.Quote].processors["feed"].pl.tail(10)

eventSymbol,bidPrice,askPrice,bidSize,askSize
str,f64,f64,f64,f64
"""BTC/USD:CXTALP""",116147.83,117327.18,0.05,0.05


In [11]:
# Example: access a specific candle interval frame (e.g., 5m)
iv = "5m"
frame = (
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .frames.get(f"{SYMBOL}{{={iv}}}")
)
frame.tail(5) if frame is not None else "No frame yet"

eventSymbol,time,eventFlags,index,sequence,count,open,high,low,close,volume,bidVolume,askVolume,openInterest,vwap,impVolatility
str,datetime[μs],i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BTC/USD:CXTALP{=5m}""",2025-08-09 17:20:00,0,7536636812328960000,0,273,116774.09,116780.1,116741.59,116742.76,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-09 17:25:00,0,7536638100819148800,0,332,116742.39,116782.69,116734.19,116775.91,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-09 17:30:00,0,7536639389309337600,0,851,116776.11,116779.63,116702.03,116707.53,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-09 17:35:00,0,7536640677799526400,0,3837,116709.2,116815.73,116708.97,116728.36,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-09 17:40:00,0,7536641966289715200,0,578,116728.35,116748.7,116711.04,116737.51,null,null,null,0.0,null,null


## Shutdown

In [12]:
await dxlink.close()
logging.info("DXLink closed. Notebook complete.")

2025-08-09 17:40:55 - INFO:tastytrade.connections.sockets:450:Listener task cancelled
2025-08-09 17:40:55 - INFO:tastytrade.connections.sockets:450:Keepalive task cancelled
2025-08-09 17:40:55 - INFO:tastytrade.connections.sockets:450:Keepalive task cancelled
2025-08-09 17:40:55 - INFO:tastytrade.connections.routing:58:Initiating cleanup...
2025-08-09 17:40:55 - INFO:tastytrade.connections.routing:82:Cleanup completed
2025-08-09 17:40:55 - INFO:tastytrade.connections.requests:175:Session closed
2025-08-09 17:40:55 - INFO:tastytrade.connections.sockets:444:Connection closed and cleaned up
2025-08-09 17:40:55 - INFO:root:2:DXLink closed. Notebook complete.
2025-08-09 17:40:55 - INFO:tastytrade.connections.routing:58:Initiating cleanup...
2025-08-09 17:40:55 - INFO:tastytrade.connections.routing:82:Cleanup completed
2025-08-09 17:40:55 - INFO:tastytrade.connections.requests:175:Session closed
2025-08-09 17:40:55 - INFO:tastytrade.connections.sockets:444:Connection closed and cleaned up
20